In [5]:
import selenium
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import io
from PIL import Image

class Webscrape:
    def __init__(self, url, filename):
        self.url=url
        self.filename=filename
        self.opts=webdriver.ChromeOptions()
        self.opts.headless=True
        self.driver = webdriver.Chrome('/usr/bin/chromedriver', options=self.opts)
        self.driver.get(url)
        self.end_of_page=self.driver.find_element_by_class_name('hg3Lgc').is_displayed()
        self.img_urls=set()
        self.startingDir=os.getcwd()
        
    def scroll(self):
        self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        button=self.driver.find_element_by_class_name('mye4qd')
        if button.is_displayed():
            button.click()
            
    def create_file(self):
        try:
            print("Creating file...")
            os.mkdir(os.path.join(os.getcwd(), self.filename))
        except:
            pass
        
    def get_image_urls(self):
        print("Getting image URLS...")
        images=self.driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        for i in tqdm(range(len(images))):
            img=images[i]
            try:
                img.click()
                act_img=self.driver.find_elements_by_css_selector('img.n3VNCb')
                for act_img in act_img:
                    if act_img.get_attribute('src') and 'https' in act_img.get_attribute('src'):
                        self.img_urls.add(act_img.get_attribute('src'))
            except:
                pass
    
    def download_images(self):
        print("Downloading images...")
        os.chdir(os.path.join(os.getcwd(), self.filename))
        baseDir=os.getcwd()
        for i, image_url in tqdm(enumerate(self.img_urls)):
            pic_name=f"{i}.jpg"
            try:
                img_content=requests.get(image_url).content
            except:
                pass
            try:
                image_file=io.BytesIO(img_content)
                image=Image.open(image_file).convert('RGB')
                file_path=os.path.join(baseDir, pic_name)
                with open(file_path, 'wb') as f:
                    image.save(f, "JPEG", quality=85)
            except:
                pass
    
    def run(self):
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        reached_page_end = False
        print('Scrolling...')
        while not reached_page_end:
            self.scroll()
            time.sleep(2)
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if last_height == new_height:
                print('End of page reached')
                break
            else:
                last_height = new_height
                
        self.create_file()
        self.get_image_urls()
        self.download_images()
        os.chdir(self.startingDir)